# Build a Mosaic of Outputs

Given outputs of different geographies (data tiles), combine these into a single layer for visualization.
In this example we take a Sentinel 2 cloudless composite for 2 months over 2 islands, Vanatu and New Caldonia and combine them into a mosaic per month.

The mosaicjson is generated and then uploaded to blob storage where it can be used by Titiler.

In [5]:
!pip install -q cogeo-mosaic

In [10]:
from os import path

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

from azure.storage.blob import ContainerClient

In [1]:
#Check access to images
!gdalinfo /vsicurl/https://deppcpublicstorage.blob.core.windows.net/output/demo/s2cloudless/s2-cloudless-sep2020-ncl.tif

Driver: GTiff/GeoTIFF
Files: /vsicurl/https://deppcpublicstorage.blob.core.windows.net/output/demo/s2cloudless/s2-cloudless-sep2020-ncl.tif
Size is 9435, 5269
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 59S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 59S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",171,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["Fals

In [7]:
sep2020_mosaic = ["https://deppcpublicstorage.blob.core.windows.net/output/demo/s2cloudless/s2-cloudless-sep2020-ncl.tif", 
                  "https://deppcpublicstorage.blob.core.windows.net/output/demo/s2cloudless/s2-cloudless-sep2020.tif"]


mosaicdata = MosaicJSON.from_urls(sep2020_mosaic)

with MosaicBackend("/home/jovyan/tmp/sep2020-mosaic.json", mosaic_def=mosaicdata) as mosaic:
    mosaic.write()

In [8]:
dec2020_mosaic = ["https://deppcpublicstorage.blob.core.windows.net/output/demo/s2cloudless/s2-cloudless-dec2020-ncl.tif", 
                  "https://deppcpublicstorage.blob.core.windows.net/output/demo/s2cloudless/s2-cloudless-dec2020.tif"]


mosaicdata = MosaicJSON.from_urls(dec2020_mosaic)

with MosaicBackend("/home/jovyan/tmp/dec2020-mosaic.json", mosaic_def=mosaicdata) as mosaic:
    mosaic.write()

In [11]:
files = ["/home/jovyan/tmp/dec2020-mosaic.json", "/home/jovyan/tmp/sep2020-mosaic.json"]
account_url = "accounturl+SAStoken"
container_client = ContainerClient.from_container_url(account_url)

for file_name in files:
    with open(file_name, "rb") as blob_file:
        blob_name = "/".join(["demo", "s2cloudless", path.basename(file_name)])
        container_client.upload_blob(name=blob_name, data=blob_file)